In [132]:
import baseline
import importlib
importlib.reload(baseline)
from baseline import *
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [177]:
#Data loading and reduce memory usage by changing dtypes
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
categories = pd.read_csv('item_categories.csv')

In [178]:
#Target - item_cnt_month
target_group = (train.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day']
                .sum().rename('item_cnt_month').reset_index())
#From EDA step we do not see linear dependency between item_cnt_month and item_price.
#Feature 'revenue' will give us more imformation about target
train['revenue'] = train['item_price'] * train['item_cnt_day']

In [135]:
'''
According to EDA: we do have a lot of data without full range during analysing period
and also our test set contains shop_id&item_id pairs that are nor presented in train set at all
'''
columns = ['date_block_num', 'shop_id', 'item_id']
full_data = full_data_creation(df = train, agg_group = columns, periods = train.date_block_num.nunique())

In [136]:
#Merge between full data and train set
full_data = full_data.merge(target_group, on = columns, how = 'left')

In [137]:
#test set concatenation with full_data
test['date_block_num'] = 34
del test['ID']
full_data = pd.concat([full_data, test], keys = columns, ignore_index=True, sort = False)

C:\Users\User\AppData\Local\Temp\ipykernel_5400\370786687.py:4: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  full_data = pd.concat([full_data, test], keys = columns, ignore_index=True, sort = False)


In [138]:
'''
We need:
1. fill all missing values as item&shop pairs + test set have been added
2. clip our target variable - original condition
'''
full_data = full_data.fillna(0)
full_data['item_cnt_month'] = full_data['item_cnt_month'].clip(0,20).astype(np.float16)

In [139]:
full_data = full_data.merge(shops, on = 'shop_id', how = 'left')
full_data = full_data.merge(items, on = 'item_id', how = 'left')
full_data = full_data.merge(categories, on = 'item_category_id', how = 'left')

In [140]:
#Columns we are planning to work
Work_columns = ['date_block_num', 'shop_id', 'item_cnt_month', 'item_id', 'city_id', 'item_category_id', 'main_category_id','minor_category_id']
full_data = full_data.loc[:, Work_columns]

#As we make transformations during DQL with shop_id, we will encode it with LabelEncoding
full_data['shop_id'] = LabelEncoder().fit_transform(full_data['shop_id'])

In [141]:
reduce_mem_usage(full_data)
full_data.info()

Mem. usage decreased to 105.42 Mb (82.8% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054182 entries, 0 to 11054181
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   date_block_num     int8   
 1   shop_id            int8   
 2   item_cnt_month     float16
 3   item_id            int16  
 4   city_id            int8   
 5   item_category_id   int8   
 6   main_category_id   int8   
 7   minor_category_id  int8   
dtypes: float16(1), int16(1), int8(6)
memory usage: 105.4 MB


In [142]:
#Check if our train data is valid
column_types = {'date_block_num': 'int8', 'shop_id': 'int8', 'city_id': 'int8', 'item_id': 'int16', 'item_cnt_month': 'float16',
'item_category_id': 'int8', 'main_category_id': 'int8', 'minor_category_id': 'int8'}
values_ranges = {'date_block_num': (0, 34), 'shop_id': (0, 59), 'item_id': (0, 22169), 'item_cnt_month': (0, 669), 'city_id':(0,30),
                'item_category_id': (0,83), 'main_category_id': (0,11), 'minor_category_id': (0, 66)}
Validator(column_types = column_types, value_ranges = values_ranges, check_missing = True, check_duplicates=True).fit_transform(full_data)

'Data is valid'

Feature exctraction

In [143]:
#From EDA we see a lot of missings in different features during the date period
#We will create feature_history with number of months (for example for shop) that feature exists
full_data = history_features(df = full_data, agg = 'shop_id', new_feature = 'shop_history')
full_data = history_features(df = full_data, agg = 'item_id', new_feature = 'item_history')
full_data = history_features(df = full_data, agg = 'minor_category_id', new_feature = 'minor_category_history')

In [144]:
#Features from aggregations
agg_list = [
    (['date_block_num', 'item_category_id'], 'avg_item_cnt_per_cat', {'item_cnt_month': 'mean'}),
    (['date_block_num', 'city_id'], 'avg_item_cnt_per_city', {'item_cnt_month': 'mean'}),
    (['date_block_num', 'shop_id'], 'avg_item_cnt_per_shop', {'item_cnt_month': 'mean'}),
    (['date_block_num', 'item_category_id', 'shop_id'], 'avg_item_cnt_per_cat_per_shop', {'item_cnt_month': 'mean'}),
    (['date_block_num', 'item_id'], 'avg_item_cnt_per_item', {'item_cnt_month': 'mean'}),
    (['date_block_num', 'item_category_id', 'shop_id'], 'med_item_cnt_per_cat_per_shop', {'item_cnt_month': 'median'}),
    (['date_block_num', 'main_category_id'], 'avg_item_cnt_per_main_cat', {'item_cnt_month': 'mean'}),
    (['date_block_num', 'minor_category_id'], 'avg_item_cnt_per_minor_cat', {'item_cnt_month': 'mean'}),
    (['date_block_num', 'shop_id', 'item_id'], 'avg_item_cnt_per_date_block', {'item_cnt_month': 'mean'}),
    (['item_id'], 'first_sales_date_block', {'item_cnt_month': 'min'})
]


for agg, new_col, aggregation in agg_list:
    full_data = feat_from_agg(full_data, agg, new_col, aggregation, full_data)


full_data['first_sales_date_block'] = full_data['first_sales_date_block'].fillna(34)

In [145]:
reduce_mem_usage(full_data)
full_data.info()

Mem. usage decreased to 358.43 Mb (50.0% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054182 entries, 0 to 11054181
Data columns (total 21 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   date_block_num                 int8   
 1   shop_id                        int8   
 2   item_cnt_month                 float16
 3   item_id                        int16  
 4   city_id                        int8   
 5   item_category_id               int8   
 6   main_category_id               int8   
 7   minor_category_id              int8   
 8   shop_history                   int8   
 9   item_history                   float16
 10  minor_category_history         int8   
 11  avg_item_cnt_per_cat           float16
 12  avg_item_cnt_per_city          float16
 13  avg_item_cnt_per_shop          float16
 14  avg_item_cnt_per_cat_per_shop  float16
 15  avg_item_cnt_per_item          float16
 16  med_item_cnt_per_cat_per_shop  float

In [146]:
#Lags of aggregational features
#All aggregations will be delted to avoid data leakage
lag_dict = {'avg_item_cnt_per_cat': [1], 'avg_item_cnt_per_shop': [1], 'avg_item_cnt_per_item': [1],
            'avg_item_cnt_per_city': [1], 'avg_item_cnt_per_cat_per_shop': [1], 
            'med_item_cnt_per_cat_per_shop': [1], 'avg_item_cnt_per_main_cat': [1],
            'avg_item_cnt_per_minor_cat': [1], 'avg_item_cnt_per_date_block': [1,2,3],
            'item_cnt_month': [1,2,3,6,12]}

for feature, lags in lag_dict.items():
    full_data = lag_features(df = reduce_mem_usage(full_data, verbose=False), col = feature, lags = lags)
    if feature != 'item_cnt_month':
        del full_data[feature]

In [151]:
full_data.columns

Index(['date_block_num', 'shop_id', 'item_cnt_month', 'item_id', 'city_id',
       'item_category_id', 'main_category_id', 'minor_category_id',
       'shop_history', 'item_history', 'minor_category_history',
       'first_sales_date_block', 'avg_item_cnt_per_cat_lag_1',
       'avg_item_cnt_per_shop_lag_1', 'avg_item_cnt_per_item_lag_1',
       'avg_item_cnt_per_city_lag_1', 'avg_item_cnt_per_cat_per_shop_lag_1',
       'med_item_cnt_per_cat_per_shop_lag_1',
       'avg_item_cnt_per_main_cat_lag_1', 'avg_item_cnt_per_minor_cat_lag_1',
       'avg_item_cnt_per_date_block_lag_1',
       'avg_item_cnt_per_date_block_lag_2',
       'avg_item_cnt_per_date_block_lag_3', 'item_cnt_month_lag_1',
       'item_cnt_month_lag_2', 'item_cnt_month_lag_3', 'item_cnt_month_lag_6',
       'item_cnt_month_lag_12', 'delta_price_lag_1'],
      dtype='object')

In [ ]:
full_data = feat_from_agg(df = train, agg = ['item_id'], new_col = 'item_avg_item_price', aggregation = {'item_price': ['mean']}, output_df = full_data)
full_data['item_avg_item_price'] = full_data['item_avg_item_price'].astype(np.float16)

full_data = feat_from_agg(df = train, agg = ['date_block_num','item_id'], new_col = 'date_item_avg_item_price', aggregation = {'item_price': ['mean']}, output_df = full_data)
full_data['date_item_avg_item_price'] = full_data['date_item_avg_item_price'].astype(np.float16)

full_data = lag_features(full_data, 'date_item_avg_item_price', [1])
full_data['delta_price_lag_1'] = (full_data['date_item_avg_item_price_lag_1'] - full_data['item_avg_item_price']) / full_data['item_avg_item_price']

del full_data['item_avg_item_price']
del full_data['date_item_avg_item_price']
del full_data['date_item_avg_item_price_lag_1']

In [170]:
#Features with last sales
full_data = last_sales(df = full_data, new_feature = 'item_shop_last_sale', item_shop = True)
full_data = last_sales(df = full_data, new_feature = 'item_last_sale', item_shop = False)

#On basis of items last sales - first sales
full_data['item_shop_first_sale'] = full_data['date_block_num'] - full_data.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
full_data['item_first_sale'] = full_data['date_block_num'] - full_data.groupby('item_id')['date_block_num'].transform('min')

In [179]:
train = train.merge(items.loc[:, ['item_id', 'item_category_id']], on = 'item_id', how = 'left')
train = train.merge(shops.loc[:, ['shop_id', 'city_id']], on = 'shop_id', how = 'left')

In [ ]:
full_data = feat_from_agg(df = train, agg = ['item_id'], new_col = 'item_avg_item_price', aggregation = {'item_price': ['mean']}, output_df = full_data)
full_data['item_avg_item_price'] = full_data['item_avg_item_price'].astype(np.float16)

full_data = feat_from_agg(df = train, agg = ['date_block_num','item_id'], new_col = 'date_item_avg_item_price', aggregation = {'item_price': ['mean']}, output_df = full_data)
full_data['date_item_avg_item_price'] = full_data['date_item_avg_item_price'].astype(np.float16)

full_data = lag_features(full_data, 'date_item_avg_item_price', [1])
full_data['delta_price_lag_1'] = (full_data['date_item_avg_item_price_lag_1'] - full_data['item_avg_item_price']) / full_data['item_avg_item_price']

del full_data['item_avg_item_price']
del full_data['date_item_avg_item_price']
del full_data['date_item_avg_item_price_lag_1']

In [161]:
#Revenue and their lags

agg_list = [
    (['date_block_num', 'item_category_id', 'shop_id'], 'sales_per_category_per_shop', {'revenue': 'sum'}),
    (['date_block_num', 'shop_id'], 'sales_per_shop', {'revenue': 'sum'}),
    (['date_block_num', 'item_id'], 'sales_per_item', {'revenue': 'sum'}),
]


for agg, new_col, aggregation in agg_list:
    full_data = feat_from_agg(train, agg, new_col, aggregation, output_df=full_data)
    

In [169]:
lag_dict = {'sales_per_category_per_shop': [1], 'sales_per_shop': [1],
            'sales_per_item': [1]}

for feature, lags in lag_dict.items():
    full_data = lag_features(df = full_data, col = feature, lags = lags)
    del full_data[feature]

In [175]:
full_data.columns

Index(['date_block_num', 'shop_id', 'item_cnt_month', 'item_id', 'city_id',
       'item_category_id', 'main_category_id', 'minor_category_id',
       'shop_history', 'item_history', 'minor_category_history',
       'first_sales_date_block', 'avg_item_cnt_per_cat_lag_1',
       'avg_item_cnt_per_shop_lag_1', 'avg_item_cnt_per_item_lag_1',
       'avg_item_cnt_per_city_lag_1', 'avg_item_cnt_per_cat_per_shop_lag_1',
       'med_item_cnt_per_cat_per_shop_lag_1',
       'avg_item_cnt_per_main_cat_lag_1', 'avg_item_cnt_per_minor_cat_lag_1',
       'avg_item_cnt_per_date_block_lag_1',
       'avg_item_cnt_per_date_block_lag_2',
       'avg_item_cnt_per_date_block_lag_3', 'item_cnt_month_lag_1',
       'item_cnt_month_lag_2', 'item_cnt_month_lag_3', 'item_cnt_month_lag_6',
       'item_cnt_month_lag_12', 'delta_price_lag_1',
       'sales_per_category_per_shop_lag_1', 'sales_per_shop_lag_1',
       'sales_per_item_lag_1', 'item_shop_last_sale', 'item_last_sale',
       'item_shop_first_sale',

In [180]:
#As for the item_price - delta_revenue
full_data = feat_from_agg(df = train, agg = ['shop_id'], new_col = 'avg_sales_per_shop', aggregation = {'revenue': ['mean']}, output_df = full_data)
full_data['avg_sales_per_shop'] = full_data['avg_sales_per_shop'].astype(np.float32)

full_data['delta_revenue_lag_1'] = (full_data['sales_per_shop_lag_1'] - full_data['avg_sales_per_shop']) / full_data['avg_sales_per_shop']

del full_data['avg_sales_per_shop']
del full_data['sales_per_shop_lag_1']


In [221]:
check = full_data.copy()
del check['item_shop_last_sale']

In [223]:
# Create a unique key for each (item_id, shop_id) combination
check['item_shop_key'] = check['item_id'].astype(str) + '_' + check['shop_id'].astype(str)

# Sort by date to ensure proper order for the lagged calculation
check = check.sort_values(by=['item_shop_key', 'date_block_num'])
check.head(5)

c:\Users\User\OneDrive\Рабочий стол\Trainee\venv\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
c:\Users\User\OneDrive\Рабочий стол\Trainee\venv\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,date_block_num,shop_id,item_cnt_month,item_id,city_id,item_category_id,main_category_id,minor_category_id,shop_history,item_history,...,item_cnt_month_lag_6,item_cnt_month_lag_12,delta_price_lag_1,sales_per_category_per_shop_lag_1,sales_per_item_lag_1,item_last_sale,item_shop_first_sale,item_first_sale,delta_revenue_lag_1,item_shop_key
7107409,20,0,0.0,0,0,40,6,6,34,1.0,...,NaN,NaN,NaN,NaN,NaN,-1,0,0,NaN,0_0
7113479,20,1,0.0,0,1,40,6,6,34,1.0,...,NaN,NaN,NaN,NaN,NaN,-1,0,0,NaN,0_1
7155969,20,11,0.0,0,8,40,6,6,34,1.0,...,NaN,NaN,NaN,NaN,NaN,-1,0,0,NaN,0_11
7162039,20,12,0.0,0,9,40,6,6,34,1.0,...,NaN,NaN,NaN,NaN,NaN,-1,0,0,NaN,0_12
7168109,20,13,0.0,0,10,40,6,6,34,1.0,...,NaN,NaN,NaN,NaN,NaN,-1,0,0,NaN,0_13


In [248]:
#Last sale feature
full_data['last_sale'] = full_data.groupby(['shop_id', 'item_id'])['date_block_num'].shift(1)
full_data['last_sale'] = full_data['last_sale']
# Calculate the number of months since the last sale
full_data['months_from_last_sale'] = full_data['date_block_num'] - full_data['last_sale']
# Calculate the number of months since the first sale
full_data['months_from_first_sale'] = full_data['date_block_num'] - full_data.groupby(['shop_id', 'item_id'])['date_block_num'].transform('min')
del full_data['last_sale']
full_data['months_from_last_sale'] = full_data['months_from_last_sale'].fillna(-1)

In [249]:
full_data = full_data.fillna(0)

In [ ]:
full_data.to_csv('full_data.csv', index = False)

Train/Test split

In [ ]:
#Validation
tss = TimeSeriesSplit(n_splits=3)

X_test = full_data[full_data.date_block_num == 34].drop('item_cnt_month', axis = 1)

X = full_data[full_data.date_block_num != 34].drop('item_cnt_month', axis = 1)
y = full_data[full_data.date_block_num != 34]['item_cnt_month']
tss = TimeSeriesSplit(n_splits=3)

for train_idxs, val_idxs in tss.split(X):

    X_train, X_val = X.iloc[train_idxs], X.iloc[val_idxs]
    y_train, y_val = y.iloc[train_idxs], y.iloc[val_idxs]
    

In [ ]:
#Training
X_train = full_data[~full_data.date_block_num.isin([33,34])]
y_train = X_train['item_cnt_month']
del X_train['item_cnt_month']

X_val = full_data[full_data['date_block_num']==33]
y_val = X_val['item_cnt_month']
del X_val['item_cnt_month']

X_test = full_data[full_data['date_block_num']==34].drop(columns='item_cnt_month')
X_test = X_test.reset_index()
del X_test['index']